In [164]:
import chess
import chess.engine

In [165]:
chess_pieces = {
    "r": "\u265c",  # black rook
    "n": "\u265e",  # black knight
    "b": "\u265d",  # black bishop
    "q": "\u265b",  # black queen
    "k": "\u265a",  # black king
    "p": "\u265f",  # black pawn
    "R": "\u2656",  # white rook
    "N": "\u2658",  # white knight
    "B": "\u2657",  # white bishop
    "Q": "\u2655",  # white queen
    "K": "\u2654",  # white king
    "P": "\u2659"   # white pawn
}

### Utility functions

In [166]:
def fen_to_board(fen):
    fen_parts = fen.split()
    rows = fen_parts[0].split('/')
    board = []
    for row in rows:
        board_row = []
        for c in row:
            if c.isdigit():
                board_row.extend([' '] * int(c))
            else:
                board_row.append(c)
        board.append(board_row)
    return board

def board_to_fen(board):
    fen_rows = []
    for row in board:
        empty_count = 0
        fen_row = ''
        for c in row:
            if c == ' ':
                empty_count += 1
            else:
                if empty_count > 0:
                    fen_row += str(empty_count)
                    empty_count = 0
                fen_row += c
        if empty_count > 0:
            fen_row += str(empty_count)
        fen_rows.append(fen_row)
    return '/'.join(fen_rows)

def algebraic_to_coord(move):
    col = ord(move[0]) - ord('a')
    row = int(move[1]) - 1
    return row, col

def make_move(board, move):
    start, end = move.strip().split()
    start_row, start_col = algebraic_to_coord(start)
    end_row, end_col = algebraic_to_coord(end)

    board[end_row][end_col] = board[start_row][start_col]
    board[start_row][start_col] = ' '
    return board
    
def make_move_return_fen(move, fen):
    board = fen_to_board(fen)
    updated_board = make_move(board, move)
    new_fen = board_to_fen(updated_board)

    return new_fen

def print_chessBoard(fen): 
    board = [[None for _ in range(8)] for _ in range(8)]

    fen_parts = fen.split()

    rank = 0
    file = 0
    for char in fen_parts[0]:
        if char == "/":
            rank += 1
            file = 0
        elif char.isdigit():
            file += int(char)
        else:
            board[rank][file] = char
            file += 1

    for rank in range(7, -1, -1):
        print(f"{rank + 1} {' '} {' '.join(str(piece) if piece is not None else '.' for piece in board[rank])}")
    print()
    print("    a b c d e f g h")

def parse_fen(fen):
    rows = fen.split('/')
    board = []

    for row in rows:
        board_row = []
        for char in row:
            if char.isnumeric():
                for _ in range(int(char)):
                    board_row.append(None)
            else:
                board_row.append(char)
        board.append(board_row)

    return board

#-------------------------GET WHITE PIECES FROM FEN NOTAITON   
def get_white_pieces(fen):
    board = fen.split()[0]
    white_pieces = []
    rank = 8
    file = 1
    for char in board:
        if char == '/':
            rank -= 1
            file = 1
        elif char.isdigit():
            file += int(char)
        else:
            piece = char.upper()
            if piece == 'P' or piece == 'N' or piece == 'B' or piece == 'R' or piece == 'Q' or piece == 'K':
                square = chr(ord('a') + file - 1) + str(rank)
                white_pieces.append(square)
            file += 1
    return white_pieces
#--------------------</>





In [167]:
piece_values = {
        'P': 10,
        'N': 30,
        'B': 30,
        'R': 50,
        'Q': 90,
        'K': 900,
        'p': -10,
        'n': -30,
        'b': -30,
        'r': -50,
        'q': -90,
        'k': -900
    }
def evaluate(fen):

    # Parse FEN string to get board state
    board_state = fen.split()[0]
    rows = board_state.split('/')
    white_pieces = []
    black_pieces = []
    for i, row in enumerate(rows):
        j = 0
        for symbol in row:
            if symbol.isdigit():
                j += int(symbol)
            else:
                piece_value = piece_values[symbol]
                if symbol.islower():
                    piece_value *= -1
                    black_pieces.append(piece_value)
                else:
                    white_pieces.append(piece_value)
                j += 1

    # Compute material score
    white_material_score = sum(white_pieces)
    black_material_score = sum(black_pieces)
    material_score = white_material_score - black_material_score

    # Compute pawn structure score
    white_pawns = [piece for piece in white_pieces if piece == piece_values['P']]
    black_pawns = [piece for piece in black_pieces if piece == piece_values['p']]
    white_pawn_structure = get_pawn_structure_score(white_pawns)
    black_pawn_structure = get_pawn_structure_score(black_pawns)
    pawn_structure_score = white_pawn_structure - black_pawn_structure

    # Compute king safety score
    white_king_pos = [(i, j) for i, row in enumerate(rows) for j, symbol in enumerate(row) if symbol == 'K'][0]

    black_king_pos = [(i, j) for i, row in enumerate(rows) for j, symbol in enumerate(row) if symbol == 'k'][0]
    white_king_safety = get_king_safety_score(white_king_pos, white_pieces)
    black_king_safety = get_king_safety_score(black_king_pos, black_pieces)
    king_safety_score = white_king_safety - black_king_safety

    # Combine scores
    score = material_score + pawn_structure_score + king_safety_score
    return score

def get_pawn_structure_score(pawns):
    """
    Compute a score for the pawn structure of a player.
    """
    score = 0
    for i, pawn in enumerate(pawns):
        # Add bonus for pawns on the 2nd or 7th rank
        if i in [1, 6]:
            score += 20
        # Add penalty for isolated pawns
        if i > 0 and i < 7 and not any(p in [i-1, i+1] for p in pawns):
            score -= 10
        # Add penalty for doubled pawns
        if pawns.count(pawn) > 1:
            score -= 10
    return score

def get_king_safety_score(king_pos, pieces):
    """
    Compute a score for the safety of a player's king.
    """
    score = 0
    # Check if king is in check
    if is_in_check(pieces, king_pos):
        score -= 50
    # Check if king is exposed
    if king_pos[1] in [0, 7]:
        score -= 10
    if king_pos[0] in [0, 7]:
        score -= 10
    # Check if king is surrounded by friendly pieces
    friendly_pieces = [piece for piece in pieces if piece > 0]
    if all(is_adjacent(king_pos, pos) for pos in get_piece_positions(friendly_pieces)):
        score += 20
    return score

def is_in_check(pieces, king_pos):
    """
    Check if the king is in check.
    """
    # Check for knight attacks
    knight_moves = get_knight_moves(king_pos)
    for move in knight_moves:
        if pieces.count(-piece_values['N']) > 0 and move in get_piece_positions([piece for piece in pieces if piece == -piece_values['N']]):
            return True
    # Check for pawn attacks
    pawn_moves = get_pawn_moves(king_pos)
    for move in pawn_moves:
        if pieces.count(-piece_values['P']) > 0 and move in get_piece_positions([piece for piece in pieces if piece == -piece_values['P']]):
            return True
    # Check for bishop, rook, and queen attacks
    directions = ['N', 'S', 'E', 'W', 'NE', 'NW', 'SE', 'SW']
    for direction in directions:
        pos = king_pos
        while True:
            pos = get_next_pos_in_direction(pos, direction)
            if not is_pos_on_board(pos):
                break
            if pieces.count(-piece_values['B']) > 0 and pos in get_piece_positions([piece for piece in pieces if piece == -piece_values['B']]):
                if direction in ['NE', 'NW', 'SE', 'SW']:
                    return True
                else:
                    break
            if pieces.count(-piece_values['R']) > 0 and pos in get_piece_positions([piece for piece in pieces if piece == -piece_values['R']]):
                if direction in ['N', 'S', 'E', 'W']:
                    return True
                else:
                    break
            if pieces.count(-piece_values['Q']) > 0 and pos in get_piece_positions([piece for piece in pieces if piece == -piece_values['Q']]):
                return True
    return False

def is_adjacent(pos1, pos2):
    """
    Check if two positions are adjacent.
    """
    return abs(pos1[0]-pos2[0]) <= 1 and abs(pos1[1]-pos2[1]) <= 1

def get_piece_positions(pieces):
    """
    Get the positions of all pieces on the board.
    """
    positions = []
    for i in range(8):
        for j in range(8):
            if (i, j) in [(piece // 10, piece % 10) for piece in pieces]:
                positions.append((i, j))
    return positions

def get_next_pos_in_direction(pos, direction):
    """
    Get the position of the next square in the given direction.
    """
    if direction == 'N':
        return (pos[0]-1, pos[1])
    elif direction == 'S':
        return (pos[0]+1, pos[1])
    elif direction == 'E':
        return (pos[0], pos[1]+1)
    elif direction == 'W':
        return (pos[0], pos[1]-1)
    elif direction == 'NE':
        return (pos[0]-1, pos[1]+1)
    elif direction == 'NW':
        return (pos[0]-1, pos[1]-1)
    elif direction == 'SE':
        return (pos[0]+1, pos[1]+1)
    elif direction == 'SW':
        return (pos[0]+1, pos[1]-1)
    else:
        return None

def is_pos_on_board(pos):
    """
    Check if a position is on the board.
    """
    return pos[0] >= 0 and pos[0] <= 7 and pos[1] >= 0 and pos[1] <= 7

def get_knight_moves(pos):
    """
    Get the squares that a knight can move to from a given position.
    """
    moves = []
    move_offsets = [(2, 1), (2, -1), (-2, 1), (-2, -1), (1, 2), (1, -2), (-1, 2), (-1, -2)]
    for offset in move_offsets:
        move = (pos[0]+offset[0], pos[1]+offset[1])
        if is_pos_on_board(move):
            moves.append(move)
    return moves

def get_pawn_moves(pos):
    """
    Get the squares that a pawn can move to from a given position.
    """
    moves = []
    if pos[0] == 6:
        moves.append((pos[0]-1, pos[1]))
        moves.append((pos[0]-2, pos[1]))
    else:
        moves.append((pos[0]-1, pos[1]))
    return moves



In [168]:

#------------------------------GET POSSIBLE MOVES OF A PIECE AT A GIVEN COORDINATE
def possible_moves_from_fen(fen, piece_coordinate):
    board = chess.Board(fen)
    piece_square = chess.parse_square(piece_coordinate)
    piece = board.piece_at(piece_square)

    if piece is None:
        raise ValueError(f"No piece found at the given coordinate: {piece_coordinate}")

    legal_moves = [move for move in board.legal_moves if move.from_square == piece_square]
    fen_moves = []

    for move in legal_moves:
        board.push(move)
        fen_moves.append(board.fen())
        board.pop()

    return fen_moves
#---------------------------------</> 

#----------------------------------GET ALL MOVES OF WHITE PIECES
def get_all_moves_of_white(white_pieces_coordinates, fen):
    white_moves = []
    for pieces in white_pieces_coordinates:
        moves = possible_moves_from_fen(fen, pieces)
        for mov in moves:
            if mov:
                white_moves.append(mov)
    return white_moves
#-----------------------------------</> 
def eval_fen(fen):
    piece_values = {
        'P': 10,
        'N': 30,
        'B': 30,
        'R': 50,
        'Q': 90,
        'K': 900,
        'p': -10,
        'n': -30,
        'b': -30,
        'r': -50,
        'q': -90,
        'k': -900
    }

    score = 0

    board_state = fen.split()[0]
    rows = board_state.split('/')
    for i, row in enumerate(rows):
        j = 0
        for symbol in row:
            if symbol.isdigit():
                j += int(symbol)
            else:
                value = piece_values[symbol]
                if symbol.islower():
                    value *= -1
                score += value
                j += 1
    return score



fen_notation = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
#print(evaluate(fen_notation))
white_p = get_white_pieces(fen_notation)
print(white_p)


['a8', 'b8', 'c8', 'd8', 'e8', 'f8', 'g8', 'h8', 'a7', 'b7', 'c7', 'd7', 'e7', 'f7', 'g7', 'h7', 'a2', 'b2', 'c2', 'd2', 'e2', 'f2', 'g2', 'h2', 'a1', 'b1', 'c1', 'd1', 'e1', 'f1', 'g1', 'h1']


In [169]:
class chessNode:
    def __init__(self, fen):
        self.fen = fen
        self.chessNodes = []
    def print_self(self):
        for nodes in self.chessNodes:
            print(nodes.fen)
    
def buildTree(fen, depth):
    node = chessNode(fen)
    if depth == 0:
        return node
    moves = get_all_moves_of_white(get_white_pieces(fen), fen)
    for move in moves:
        child_node = buildTree(move, depth - 1)
        node.chessNodes.append(child_node)
    return node

node = buildTree(fen=fen_notation, depth=3)


In [181]:
def minimax(node, depth, alpha, beta, maximizing_player, evaluation_function):
    """
    Perform the minimax algorithm with alpha-beta pruning on a given game tree node.
    """
    if depth == 0 or len(node.chessNodes) == 0:
        return evaluation_function(node.fen)
    
    if maximizing_player:
        value = -float('inf')
        for child_node in node.chessNodes:
            value = max(value, minimax(child_node, depth-1, alpha, beta, False, evaluation_function))
            alpha = max(alpha, value)
            if beta <= alpha:
                break
        return value
    else:
        value = float('inf')
        for child_node in node.chessNodes:
            value = min(value, minimax(child_node, depth-1, alpha, beta, True, evaluation_function))
            beta = min(beta, value)
            if beta <= alpha:
                break
        return value



def get_best_move():
    best_move = None
    best_value = -float('inf')
    for child_node in node.chessNodes:
        value = minimax(child_node, depth=2, alpha=-float('inf'), beta=float('inf'), maximizing_player=False, evaluation_function=evaluate)
        if value > best_value:
            best_move = child_node.fen
            best_value = value
    return best_move, best_value

best_move, best_value = get_best_move()
print(best_move)



rnbqkbnr/pppppppp/8/8/8/3P4/PPP1PPPP/RNBQKBNR b KQkq - 0 1
